In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('data.csv', encoding = 'ISO-8859-1')
df.head()

In [ ]:
def plot_top_products(dataframe):
    # Group the data by product and sum the quantity
    products = dataframe.groupby('Description').sum(numeric_only=False)['Quantity']
    # Sort the products by quantity
    products = products.sort_values(ascending=False)
    # Plot the top 10 products
    products[:10].plot(kind='bar')
    plt.xlabel('Product')
    plt.ylabel('Quantity Sold')
    plt.title('Top 10 Selling Products')
    plt.show()

In [ ]:
def plot_top_returned_products(dataframe):
    # Only get rows where quantity is less than zero
    dataframe = dataframe[dataframe['Quantity'] < 0]
    # Group by product and sum the quantity
    products = dataframe.groupby('Description').sum(numeric_only=False)['Quantity']
    # Sort products by quantity, but since it's negative we sort in ascending order
    products = products.sort_values(ascending=True)
    # Plot the top 10 returned products
    ax = products[:10].plot(kind='bar')
    ax.invert_yaxis()
    plt.xlabel('Quantity Returned')
    plt.ylabel('Product')
    plt.title('Top 10 Returned Products')
    plt.show()

In [ ]:
def plot_top_customers(dataframe):
    # Group the data by customer and sum the Quantity
    customers = dataframe.groupby('CustomerID').sum(numeric_only=False)['Quantity']
    # Sort the customers by quantity
    customers = customers.sort_values(ascending=False)
    # Plot the top 10 customers
    customers.head(10).plot(kind='bar')
    plt.xlabel('Customer ID')
    plt.ylabel('Quantity Purchased')
    plt.title('Top 10 Customers by Quantity Purchased')
    plt.show()

In [ ]:
def plot_top_returned_customers(dataframe):
    # Group by customer and sum the Quantity
    products = dataframe.groupby('CustomerID').sum(numeric_only=False)['Quantity']
    # Sort customer by quantity, but since it's negative we sort in ascending order
    products = products.sort_values(ascending=True)
    # Plot the top 10 returned customers
    ax = products[:10].plot(kind='bar')
    ax.invert_yaxis()
    plt.xlabel('Quantity Returned')
    plt.ylabel('Customers')
    plt.title('Top 10 Returned Customers')
    plt.show()

In [ ]:
def plot_top_products_by_country(dataframe):
    # Only get rows where quantity is greater than zero (to ignore returns)
    dataframe = dataframe[dataframe['Quantity'] > 0]
    countries = dataframe['Country'].unique()
    for country in countries:
        # Filter data to include only rows for current country
        df_country = dataframe[dataframe['Country'] == country]
        # Group by product and sum the quantity
        products = df_country.groupby('Description').sum(numeric_only=False)['Quantity']
        # Sort products by quantity
        products = products.sort_values(ascending=False)
        # Plot the top 10 products
        products[:10].plot(kind='bar')
        plt.xlabel('Product')
        plt.ylabel('Quantity Sold')
        plt.title('Top 10 Sold Products in {}'.format(country))
        plt.show()